In [1]:
import csv
from collections import namedtuple
from itertools import combinations

import numpy as np
import pandas as pd
import pickle
from src.utils import check_hash, haversine_km

In [2]:
# Check Excel file hash (in case we accidentally changed some data in the file).
POSM_FILE = "./data/posm_post_office_coords.xlsx"
assert check_hash(POSM_FILE)

# Import the Excel file. Keep the Postcode column as a string.
df_import = pd.read_excel(POSM_FILE, sheet_name="Sheet2", dtype={"POSTCODE": str})

In [3]:
df = df_import.rename(
    columns={
        "LOCATION": "location",
        "POSTCODE": "postcode",
        "POST_OFFICE": "district_0",
        "POST_OFFICE_1": "district_1",
        "STATE": "state",
        "DATEUPDATE": "last_updated",
        "POINT_X": "longitude",
        "POINT_Y": "latitude",
    }
)
display(df.head())
display(df.info())

,location,postcode,district_0,district_1,state,last_updated,longitude,latitude
0,Pusat Komersial & Perindustrian Gangsa Jaya,76100,Melaka,Melaka,Melaka,2020-10-02 00:00:00.0000000,102.264939,2.214094
1,Rimbun Kiara,70200,Seremban,Seremban,Negeri Sembilan,2020-10-02 00:00:00.0000000,101.941512,2.719340
2,Rimbun Impian,70300,Seremban,Seremban,Negeri Sembilan,2020-10-02 00:00:00.0000000,101.941512,2.719340
3,Rimbun Jasmin,70300,Seremban,Seremban,Negeri Sembilan,2020-10-02 00:00:00.0000000,101.941512,2.719340
4,Nusari Bayu 2,71950,Seremban,Seremban,Negeri Sembilan,2020-10-02 00:00:00.0000000,101.941512,2.719340


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71031 entries, 0 to 71030
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   location      71031 non-null  object 
 1   postcode      71031 non-null  object 
 2   district_0    71031 non-null  object 
 3   district_1    71031 non-null  object 
 4   state         71031 non-null  object 
 5   last_updated  71031 non-null  object 
 6   longitude     71031 non-null  float64
 7   latitude      71031 non-null  float64
dtypes: float64(2), object(6)
memory usage: 4.3+ MB


None

# Cleanup

In [4]:
# White space and capitalisation.
for col in ["location", "postcode", "district_0", "district_1", "state"]:
    df[col] = df[col].apply(lambda x: x.strip().removesuffix("_x000D_"))
    df[col] = df[col].apply(lambda x: x[0].upper() + x[1:])

# We can drop location and date_updated, as it's not very useful.
df = df.drop(columns=["location", "last_updated"]).drop_duplicates()

# Combine longitude and latitude into 1 column.
# It makes for easier aggregate operations later.
df["lat_lon"] = list(zip(df["latitude"], df["longitude"]))
df = df.drop(columns=["latitude", "longitude"])

# Clean state column.
CLEANED_STATES = [
    "Johor",
    "Kedah",
    "Kelantan",
    "Melaka",
    "Negeri Sembilan",
    "Pahang",
    "Perak",
    "Perlis",
    "Pulau Pinang",
    "Sabah",
    "Sarawak",
    "Selangor",
    "Terengganu",
    "WP Kuala Lumpur",
    "WP Labuan",
    "WP Putrajaya",
]


def unclean_states():
    return sorted([s for s in df.state.unique() if s not in CLEANED_STATES])

print("Unclean states:", unclean_states())

clean_state_dict = {
    "WP kuala Lumpur": "WP Kuala Lumpur",
}

df.state = df.state.apply(
    lambda x: clean_state_dict[x] if x in clean_state_dict.keys() else x
)
print("Unclean states:", unclean_states())
assert unclean_states() == []

# Why are there two district fields? After cleaning, they are the same.
df_districts = df.loc[:, ["district_0", "district_1"]].drop_duplicates()
for col in df_districts:
    print(f"{col} has {len(df_districts[col].unique())} unique values.")
assert len(df_districts.district_0.unique()) == len(df_districts.district_0.unique())

# We can drop district_0 and rename district_1 to district.
df = df.drop(columns="district_0").rename(columns={"district_1": "district"})
df.head()

# Add PPV regions: KL, Selangor & Putrajaya are the same region
df["ppv_region"] = df["state"].apply(
    lambda x: "Selangor/KL/Putrajaya"
    if x in ["Selangor", "WP Kuala Lumpur", "WP Putrajaya"]
    else x
)

Unclean states: ['WP kuala Lumpur']
Unclean states: []
district_0 has 437 unique values.
district_1 has 437 unique values.


In [5]:
# Unique data per column
for c in df.columns:
    print(c, len(df[c].unique()))

postcode 2886
district 437
state 16
lat_lon 419
ppv_region 14


# Investigation: Bad postcodes

In [6]:
# TODO: What do we do with these "bad" postcodes?

df["postcode_2digits"] = df["postcode"].apply(lambda x: x[:2])
temp = df[["postcode_2digits", "state"]].drop_duplicates()

# Note that two of these "two-digit postcodes" span multiple states
temp = temp["postcode_2digits"].value_counts()
temp = temp[temp>1]

print("These 2-digit postcodes span multiple states:")
display(temp)
print("-----")

# Retrieve all full postcodes with this problem
bad_postcodes = temp.index.to_list()
bad_rows_idx = df[df["postcode_2digits"].isin(bad_postcodes)].index.to_list()
df_bad = df.loc[bad_rows_idx].sort_values("postcode")
print(f"There are {len(df_bad)} bad postcodes.")
display(df_bad)
del df["postcode_2digits"]

These 2-digit postcodes span multiple states:


34    2
14    2
Name: postcode_2digits, dtype: int64

-----
There are 37 bad postcodes.


,postcode,district,state,lat_lon,ppv_region,postcode_2digits
668,14000,Bukit Mertajam,Pulau Pinang,"(5.365022, 100.459264)",Pulau Pinang,14
1455,14007,Bukit Mertajam,Pulau Pinang,"(5.365022, 100.459264)",Pulau Pinang,14
1456,14009,Bukit Mertajam,Pulau Pinang,"(5.365022, 100.459264)",Pulau Pinang,14
1457,14020,Bukit Mertajam,Pulau Pinang,"(5.365022, 100.459264)",Pulau Pinang,14
600,14100,Simpang Ampat,Pulau Pinang,"(5.283955, 100.476943)",Pulau Pinang,14
1913,14110,Simpang Ampat,Pulau Pinang,"(5.283955, 100.476943)",Pulau Pinang,14
1921,14120,Simpang Ampat,Pulau Pinang,"(5.283955, 100.476943)",Pulau Pinang,14
601,14200,Sungai Jawi,Pulau Pinang,"(5.215871, 100.497033)",Pulau Pinang,14
649,14300,Nibong Tebal,Pulau Pinang,"(5.167157, 100.475425)",Pulau Pinang,14
1805,14310,Nibong Tebal,Pulau Pinang,"(5.167157, 100.475425)",Pulau Pinang,14


# Master postcode

We map every postcode to it's "master postcode": a similar postcode with the same lat_long coordinates, but with a lower level. If there are multiple candidates for the master postcode, pick the smaller postcode (when treated as an integer).

Example: if 54000, 54100, 54120, 60000, 61000, 61900, 61990 and 70000 all share the same coordinates, we map all of them to the same "master postcode": 60000 (a 1st-level postcode).

In [7]:
# Helper function t
def postcode_level(postcode: str) -> int:
    """Returns the postcode level: number of digits before the ending string of zeroes.

    e.g. 54000 -> Level 2 postcode
         54680 -> Level 4 postcode
    """
    return len(postcode.rstrip("0"))

Unfortunately, there are some postcodes that each have multiple coordinates...

In [8]:
# Find all postcodes which have multiple lat_lon coordinates
temp = (
    df[["postcode", "lat_lon"]]
    .groupby("postcode")
    .count()
    .rename(columns={"lat_lon": "num_lat_lon"})
)
temp = temp[temp["num_lat_lon"] > 1].sort_values(by="num_lat_lon", ascending=False)
display(temp)
print("-----")
print("Example: postcode 21400:")
display(df[df["postcode"] == "21400"])

,num_lat_lon
postcode,
21400,3
06010,2
57000,2
81300,2
76100,2
75460,2
72120,2
71800,2
59200,2


-----
Example: postcode 21400:


,postcode,district,state,lat_lon,ppv_region
4908,21400,Bukit Payong,Terengganu,"(5.232199, 103.101954)",Terengganu
14255,21400,Kuala Terengganu,Terengganu,"(5.337306, 103.13771)",Terengganu
61939,21400,Chalok,Terengganu,"(5.424328, 102.837014)",Terengganu


For these postcodes, use the coordinates with the smallest latitude.

We could also use the average coordinate, but there is no guarantee that this coordinate is near a road. So let's stick with smallest latitude.

In [9]:
# Pick the smallest coordinates (tuple comparison always compares latitude first)
df["lat_lon"] = df[["postcode", "lat_lon"]].groupby("postcode").transform(np.min)
display(df[df["postcode"] == "21400"])
# Change above to np.mean for the average position instead

,postcode,district,state,lat_lon,ppv_region
4908,21400,Bukit Payong,Terengganu,"(5.232199, 103.101954)",Terengganu
14255,21400,Kuala Terengganu,Terengganu,"(5.232199, 103.101954)",Terengganu
61939,21400,Chalok,Terengganu,"(5.232199, 103.101954)",Terengganu


Now, we create a mapping of all postcodes to their master postcode.

In [10]:
# Create mapping of all postcodes to their master postcode

# Master postcode is the postcode with the most zeroes at the end. Break ties (e.g. 50000, 60000) by picking the smaller number.
def master_postcode_sort(postcode: str):
    return (postcode_level(postcode), int(postcode))

def master_postcode_agg(series):
    postcodes = series.to_list()
    return min(postcodes, key=master_postcode_sort)

df["master_postcode"] = (
    df[["postcode", "lat_lon"]].
    groupby(["lat_lon"]).
    transform(master_postcode_agg)
)
display(df.head())


,postcode,district,state,lat_lon,ppv_region,master_postcode
0,76100,Melaka,Melaka,"(2.214094, 102.264939)",Melaka,75000
1,70200,Seremban,Negeri Sembilan,"(2.71934, 101.941512)",Negeri Sembilan,70000
2,70300,Seremban,Negeri Sembilan,"(2.71934, 101.941512)",Negeri Sembilan,70000
4,71950,Seremban,Negeri Sembilan,"(2.71934, 101.941512)",Negeri Sembilan,70000
5,43000,Kajang,Selangor,"(2.99319, 101.787)",Selangor/KL/Putrajaya,43000


In [11]:
# dataframe_postcodes
df_pc = df[
    ["postcode", "master_postcode", "state", "ppv_region", "lat_lon"]
].drop_duplicates()

display(df_pc)

,postcode,master_postcode,state,ppv_region,lat_lon
0,76100,75000,Melaka,Melaka,"(2.214094, 102.264939)"
1,70200,70000,Negeri Sembilan,Negeri Sembilan,"(2.71934, 101.941512)"
2,70300,70000,Negeri Sembilan,Negeri Sembilan,"(2.71934, 101.941512)"
4,71950,70000,Negeri Sembilan,Negeri Sembilan,"(2.71934, 101.941512)"
5,43000,43000,Selangor,Selangor/KL/Putrajaya,"(2.99319, 101.787)"
...,...,...,...,...,...
70583,88875,88000,Sabah,Sabah,"(5.982556, 116.074506)"
70584,89707,89700,Sabah,Sabah,"(5.52718611, 115.857611)"
70585,91128,91100,Sabah,Sabah,"(5.024247, 118.330774)"
70588,32100,32100,Perak,Perak,"(4.21121389, 100.641323)"


In [12]:
# dataframe_masterpostcodes
# Sort by region for nicer matrix later.
df_mpc = df_pc.drop(columns="postcode").drop_duplicates()
df_mpc = df_mpc.sort_values(by=["ppv_region", "master_postcode"]) 

display(df_mpc)

,master_postcode,state,ppv_region,lat_lon
80,79000,Johor,Johor,"(1.42513621, 103.61443042)"
51,80000,Johor,Johor,"(1.456123, 103.761701)"
4237,81000,Johor,Johor,"(1.662964, 103.600178)"
3800,81400,Johor,Johor,"(1.606506, 103.647617)"
4512,81440,Johor,Johor,"(1.876001, 103.614046)"
...,...,...,...,...
6672,24050,Terengganu,Terengganu,"(4.26868955, 103.2119044)"
6654,24100,Terengganu,Terengganu,"(4.335356, 103.479837)"
6690,24200,Terengganu,Terengganu,"(4.426911, 103.452517)"
7300,24300,Terengganu,Terengganu,"(4.50032, 103.440871)"


# Distance Matrix

Generate a distance matrix: the haversine distance between any two master postcodes. Postcodes in different `ppv_region`s will be blank (NA) as we can't travel across borders.

Since this is quite expensive computation-wise, we will also generate a csv which stores all the valid master postcode pairings and coordinates. This will be used for for feeding into Google Maps API.

In [13]:
# A list of dicts containing all valid master postcode pairings, to be converted to csv later.
mpc_pairs = []

# Generate distance matrix.
pc_labels = [(x, y) for (x, y) in zip(df_mpc["ppv_region"], df_mpc["master_postcode"])]
master_lat_lon = {x: y for (x, y) in zip(df_mpc["master_postcode"], df_mpc["lat_lon"])}

# dataframe_matrix
df_mat = pd.DataFrame(
    pd.NA, 
    index=pd.MultiIndex.from_tuples(pc_labels), 
    columns=pd.MultiIndex.from_tuples(pc_labels),
)  # type: ignore

# Diagonals are zero distance
for tup in pc_labels:
    df_mat.loc[tup, tup] = 0

# Off-diagonals, calculate the haversine distance.
for tup1, tup2 in combinations(pc_labels, 2):
    ppv1, mpc1 = tup1
    ppv2, mpc2 = tup2
    if ppv1 == ppv2:
        if mpc1 == mpc2:
            df_mat.loc[tup1, tup2] = 0
        else:
            lat1, lon1 = master_lat_lon[mpc1]
            lat2, lon2 = master_lat_lon[mpc2]
            distance = haversine_km(lat1, lon1, lat2, lon2)
            df_mat.loc[tup1, tup2] = distance
            df_mat.loc[tup2, tup1] = distance  # reflect across the diagonal
            # This is a valid pair, so add it to the list.
            mpc_pairs.append({
                "ppv_region": ppv1,
                "master_postcode_1": mpc1,
                "latitude_1": lat1,
                "longitude_1": lon1,
                "master_postcode_2": mpc2,
                "latitude_2": lat2,
                "longitude_2": lon2, 
            })


# Output files

In [14]:
# Save to Excel sheets:
with pd.ExcelWriter("./output/postcode_output.xlsx") as writer:
    df_pc.to_excel(writer, sheet_name="Postcodes")
    df_mpc.to_excel(writer, sheet_name="Master Postcodes")
    df_mat.to_excel(writer, sheet_name="Master Postcode Distance Matrix")

# Save the master postcode pairs as a csv
with open("./output/mpc_pairs.csv", mode="w", newline='') as fp:
    writer = csv.DictWriter(fp, fieldnames=mpc_pairs[0].keys())
    writer.writeheader()
    for pair in mpc_pairs:
        writer.writerow(pair)



In [15]:
with open("./output/distance_matrix.pickle", mode="wb") as fp:
    pickle.dump(df_mat, fp)

# Conclusion

In [16]:
num = len(mpc_pairs)
print(f"There are a total of {num} valid master postcode pairs.")
print(f"Total cost in Maps API: USD{num * 0.005}.")
print("-----")
print(f"You can find the data output files in the `output` folder.")


There are a total of 8332 valid master postcode pairs.
Total cost in Maps API: USD41.660000000000004.
-----
You can find the data output files in the `output` folder.
